In [1]:
import pandas as pd
import re
import numpy as np
import os

In [2]:
os.chdir(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE")

# ---------- PATH SETUP ----------
s2_path = os.path.join('dataset', 'annotations', 'pone.0231608.s002.xlsx')
s3_path = os.path.join('dataset', 'annotations', 'pone.0231608.s003.xlsx')

os.makedirs(os.path.join('outputs', 'csv'), exist_ok=True)

# ---------- S2: Extract Unique Codes ----------
s2_df = pd.read_excel(s2_path, sheet_name='FACS')

# Detect column containing codes (robust to differences)
possible_cols = [c for c in s2_df.columns if str(c).lower() in ['code', 'facs code', 'au code', 'action unit', 'au']]
if possible_cols:
    code_col = possible_cols[0]
else:
    for c in s2_df.columns:
        if s2_df[c].astype(str).str.contains(r'(AU|AD|VC|EAD)', regex=True).any():
            code_col = c
            break
    else:
        code_col = s2_df.columns[0]

print(f"Detected Code column in S2 → {code_col}")

s2_codes = set(
    s2_df[code_col]
    .dropna()
    .astype(str)
    .str.strip()
    .unique()
)

# ---------- S3: Extract Codes from Media Sections & Tiers ----------
s3_raw = pd.read_excel(s3_path, sheet_name='FACS', header=None)
s3_codes = set()
s3_tier_groups = {}

current_tier = None
media_start = 0

for idx, row in s3_raw.iterrows():
    cell0 = str(row[0]).strip() if pd.notna(row[0]) else ''

    if cell0.startswith("Media"):
        # Process block up to previous Media
        if idx > media_start:
            section_df = s3_raw.iloc[media_start:idx].dropna(how="all")
            text = " ".join(section_df.astype(str).values.flatten())
            found_codes = re.findall(r'(AU\d+|AD\d+|VC\d+|EAD\d+|Non-scoreable|Uncodable)', text)
            s3_codes.update(found_codes)
            if current_tier and found_codes:
                s3_tier_groups[current_tier] = list(set(found_codes))

        media_start = idx + 1
        tier_row = media_start + 2
        if tier_row < len(s3_raw):
            row_values = [str(v).strip() for v in list(s3_raw.iloc[tier_row].dropna())]
            current_tier = row_values[0] if row_values else None

# Final block
section_df = s3_raw.iloc[media_start:].dropna(how="all")
text = " ".join(section_df.astype(str).values.flatten())
found_codes = re.findall(r'(AU\d+|AD\d+|VC\d+|EAD\d+|Non-scoreable|Uncodable)', text)
s3_codes.update(found_codes)
if current_tier and found_codes:
    s3_tier_groups[current_tier] = list(set(found_codes))

# ---------- Combine Unique Codes ----------
all_codes = sorted(list(s2_codes.union(s3_codes)))

# ---------- Region Inference ----------
def infer_region(code):
    code_upper = code.upper()
    if re.match(r'EAD\d+', code_upper):
        return 'Ears'
    elif re.match(r'VC\d+', code_upper):
        return 'Visibility Code'
    elif re.match(r'AD5[1-8]', code_upper):
        return 'Head Movement'
    elif re.match(r'(AU101|AU5|AU145|AU47|AD1)', code_upper):
        return 'Upper Face (Eyes)'
    elif re.match(r'(AD38|AU10|AU113|AU17|AU18|AU24|AD19)', code_upper):
        return 'Lower Face (Nostrils/Chin)'
    elif code in ['Non-scoreable', 'Uncodable']:
        return 'Uncodable'
    else:
        return 'Other'

combined_df = pd.DataFrame([{"Code": c, "Region": infer_region(c)} for c in all_codes])

print("\n=== Combined Unique Codes + Inferred Regions ===")
print(combined_df.to_string(index=False))

# ---------- Print S3 Tier Groups ----------
print("\n=== S3 Codes by Tier ===")
for tier, codes in sorted(s3_tier_groups.items()):
    print(f"{tier}: {', '.join(sorted(codes))}")

# ---------- Save Outputs ----------
combined_df.to_csv(os.path.join('outputs', 'csv', 'equifacs_codes_combined.csv'), index=False)
tier_df = pd.DataFrame(
    [(tier, ", ".join(sorted(codes))) for tier, codes in s3_tier_groups.items()],
    columns=['Tier', 'Codes']
)
tier_df.to_csv(os.path.join('outputs', 'csv', 'equifacs_codes_s3_tiers.csv'), index=False)

print("\n✔ CSVs saved:")
print("  • outputs/csv/equifacs_codes_combined.csv")
print("  • outputs/csv/equifacs_codes_s3_tiers.csv")


C:\Users\hp\AppData\Local\Temp\ipykernel_14696\1311144869.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if s2_df[c].astype(str).str.contains(r'(AU|AD|VC|EAD)', regex=True).any():
C:\Users\hp\AppData\Local\Temp\ipykernel_14696\1311144869.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if s2_df[c].astype(str).str.contains(r'(AU|AD|VC|EAD)', regex=True).any():


Detected Code column in S2 → FILM 1 

=== Combined Unique Codes + Inferred Regions ===
         Code                     Region
          AD1          Upper Face (Eyes)
        AD133          Upper Face (Eyes)
        AD160          Upper Face (Eyes)
         AD19          Upper Face (Eyes)
         AD1L          Upper Face (Eyes)
         AD29                      Other
         AD30                      Other
         AD38 Lower Face (Nostrils/Chin)
         AD51              Head Movement
         AD52              Head Movement
         AD53              Head Movement
         AD54              Head Movement
         AD55              Head Movement
         AD56              Head Movement
         AD57              Head Movement
         AD58              Head Movement
         AD76                      Other
         AD81                      Other
         AD84                      Other
         AD85                      Other
         AU10 Lower Face (Nostrils/Chin)
        AU1